In [1]:
from __future__ import print_function
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
from collections import Counter, OrderedDict

from PIL import Image as PImage
from os import listdir
from pickle import dump
import matplotlib.pyplot as plt
import PIL, cv2, os, json, glob, h5py, keras, csv
from IPython.display import SVG

import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical
from keras.backend.tensorflow_backend import set_session
from keras.callbacks import TensorBoard
from keras.applications.vgg16 import preprocess_input
from keras.utils.vis_utils import plot_model, model_to_dot

#import seaborn as sns

Using TensorFlow backend.


In [2]:
!pip show tensorflow

Name: tensorflow
Version: 1.8.0
Summary: TensorFlow helps the tensors flow
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /home/wayne/anaconda3/lib/python3.6/site-packages
Requires: six, protobuf, grpcio, absl-py, termcolor, wheel, astor, numpy, gast, tensorboard
Required-by: 


In [3]:
train_path = '../data/aia-picture-classification1/train'
test_path = '../data/aia-picture-classification1/test'
model_path = '../data/aia-picture-classification1/model'

In [4]:
import tensorflow as tf

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18121502417541566786
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15870492672
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3258721779488506173
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [6]:
label_list = !ls {train_path}
label_list

['bedroom',
 'CALsuburb',
 'coast',
 'forest',
 'highway',
 'industrial',
 'insidecity',
 'kitchen',
 'livingroom',
 'mountain',
 'opencountry',
 'PARoffice',
 'store',
 'street',
 'tallbuilding']

In [7]:
mtype_dict = {}
mtype_list = [None] * len(label_list)
with open('../data/aia-picture-classification1/target_to_number.txt', newline='') as csvfile:
    _dict = csv.DictReader(csvfile)
    for idx, row in enumerate(_dict):
        mtype_dict[row['type'].strip()] = int(row[' index'].strip())
        mtype_list[int(row[' index'].strip())] = row['type'].strip()

In [8]:
mtype_list

['kitchen',
 'street',
 'industrial',
 'insidecity',
 'forest',
 'livingroom',
 'opencountry',
 'PARoffice',
 'mountain',
 'CALsuburb',
 'coast',
 'store',
 'bedroom',
 'tallbuilding',
 'highway']

In [9]:
from keras.layers import *
from keras.optimizers import *
from keras.applications import *

from keras.preprocessing.image import img_to_array, load_img
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Conv2D, Dropout, BatchNormalization
from keras import regularizers
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.applications.xception import Xception, preprocess_input

In [10]:
img_width , img_height = 299, 299  # change based on the shape/structure of your images
nb_classes = len(mtype_dict)
batch_size = 32
nb_epoch = 100
based_model_last_block_layer_number = 126  # value is based on based model selected.

In [11]:
base_model = Xception(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)

for layer in base_model.layers:
    layer.trainable = False

    
x = base_model.output
x = GlobalMaxPooling2D()(x)
x = Dropout(0.2)(x)
output = Dense(nb_classes, activation='softmax')(x)    

# add your top layer block to your base model
model = Model(base_model.input, output)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [12]:
batch_size

32

In [13]:
data_gen = image.ImageDataGenerator(rescale=1. / 255, 
                                    validation_split=0.1,
                                    featurewise_center=True,
                                    fill_mode="nearest",
                                    preprocessing_function=preprocess_input)

In [14]:
validation_generator = data_gen.flow_from_directory(train_path,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True, seed=13, subset="validation")

Found 293 images belonging to 15 classes.


In [15]:
data_gen.featurewise_center=True
data_gen.rotation_range=20
data_gen.width_shift_range=0.2
data_gen.height_shift_range=0.2
data_gen.shear_range=0.2
data_gen.zoom_range=[0.6, 1.4]
data_gen.horizontal_flip=True

In [16]:
train_generator = data_gen.flow_from_directory(train_path,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True, seed=13, subset="training")

Found 2692 images belonging to 15 classes.


In [31]:
test_datagen = image.ImageDataGenerator(rescale=1./255, featurewise_center=True, fill_mode="nearest",
                                       preprocessing_function=preprocess_input)

test_generator =  test_datagen.flow_from_directory(test_path, 
                                     target_size=(img_width, img_height), 
                                     batch_size=batch_size,
                                     shuffle=False,
                                     class_mode=None)

Found 1500 images belonging to 1 classes.


In [17]:
model.compile(optimizer='nadam',
              loss=categorical_crossentropy, metrics=['accuracy',])

In [18]:
top_weights_path = os.path.join(model_path, 'top_model_weights_7.h5')
csv_path = os.path.join(model_path, 'top_model_csv_7.h5')
callbacks_list = [
    ModelCheckpoint(top_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
    EarlyStopping(monitor='loss', patience=10, verbose=0),
    CSVLogger(csv_path, separator=',', append=False)
]
#tensor_board = callbacks.TensorBoard()

In [19]:
filenames = train_generator.filenames
nb_samples = len(filenames)

print('There are {0} training data.'.format(train_generator.n))

There are 2692 training data.


In [20]:
nb_epoch // 5

20

In [21]:
model.fit_generator(train_generator,
                    epochs=nb_epoch / 5,
                    validation_data=validation_generator,
                    callbacks=callbacks_list,
                    workers=8,
                    use_multiprocessing=True)

Epoch 1/20
85/85 [==============================] - 63s 737ms/step - loss: 3.0400 - acc: 0.3309 - val_loss: 5.6180 - val_acc: 0.0922

Epoch 00001: val_acc improved from -inf to 0.09215, saving model to ../data/aia-picture-classification1/model/top_model_weights_7.h5
Epoch 2/20
85/85 [==============================] - 56s 654ms/step - loss: 1.6390 - acc: 0.5964 - val_loss: 6.7152 - val_acc: 0.0887

Epoch 00002: val_acc did not improve from 0.09215
Epoch 3/20
21/85 [======>.......................] - ETA: 25s - loss: 1.3873 - acc: 0.6161

Process ForkPoolWorker-34:
Process ForkPoolWorker-38:
Process ForkPoolWorker-43:
KeyboardInterrupt
Process ForkPoolWorker-33:
Traceback (most recent call last):
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
Process ForkPoolWorker-44:
  File "/home/wayne/anaconda3/lib/python3.6/multiprocessing/process.py", line 258,

  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 1737, in _get_batches_of_transformed_samples
    x = self.image_data_generator.apply_transform(x, params)
KeyboardInterrupt
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 1114, in apply_transform
    fill_mode=self.fill_mode, cval=self.cval)
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 325, in <listcomp>
    cval=cval) for x_channel in x]
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py", line 325, in apply_affine_transform
    cval=cval) for x_channel in x]
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_applications/xception.py", line 338, in preprocess_input
    return imagenet_utils.preprocess_input(x, mode='tf')
  File "/home/wayne/anaconda3/lib/python3.6/site-packages/keras_applications/imagenet_utils.py", line 189, in preprocess_input
    

KeyboardInterrupt: 

In [23]:
model.load_weights(top_weights_path)

In [24]:
for layer in model.layers[:based_model_last_block_layer_number]:
    layer.trainable = False
for layer in model.layers[based_model_last_block_layer_number:]:
    layer.trainable = True

In [25]:
model.compile(optimizer='nadam',
              loss=categorical_crossentropy, metrics=['accuracy',])

In [26]:
final_weights_path = os.path.join(model_path, 'model_weights_v7.h5')
csv_path = os.path.join(model_path, 'model_csv_v7.csv')
callbacks_list = [
    ModelCheckpoint(final_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
    EarlyStopping(monitor='loss', patience=20, verbose=0),
    CSVLogger(csv_path, separator=',', append=False)
]

In [27]:
# fine-tune the model
model.fit_generator(train_generator,
                    epochs=nb_epoch,
                    validation_data=validation_generator,
                    callbacks=callbacks_list,
                    workers=8,
                    use_multiprocessing=True)

# save model
model_json = model.to_json()
with open(os.path.join(model_path, 'model_v7.json'), 'w') as json_file:
    json_file.write(model_json)

Epoch 1/100
85/85 [==============================] - 61s 723ms/step - loss: 0.9815 - acc: 0.7552 - val_loss: 0.9778 - val_acc: 0.7304

Epoch 00001: val_acc improved from -inf to 0.73038, saving model to ../data/aia-picture-classification1/model/model_weights_v7.h5
Epoch 2/100
85/85 [==============================] - 54s 630ms/step - loss: 0.4234 - acc: 0.8617 - val_loss: 0.8429 - val_acc: 0.7747

Epoch 00002: val_acc improved from 0.73038 to 0.77474, saving model to ../data/aia-picture-classification1/model/model_weights_v7.h5
Epoch 3/100
85/85 [==============================] - 54s 636ms/step - loss: 0.3509 - acc: 0.8835 - val_loss: 1.0632 - val_acc: 0.7167

Epoch 00003: val_acc did not improve from 0.77474
Epoch 4/100
85/85 [==============================] - 55s 641ms/step - loss: 0.2997 - acc: 0.8948 - val_loss: 0.8082 - val_acc: 0.7509

Epoch 00004: val_acc did not improve from 0.77474
Epoch 5/100
85/85 [==============================] - 54s 635ms/step - loss: 0.2632 - acc: 0.9103 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
model.load_weights(final_weights_path)

In [32]:
X_l_vec = model.predict_generator(generator=test_generator)

In [34]:
predict_index = np.argmax(X_l_vec, axis=-1)

In [35]:
fnames = [_f.split('/')[1].replace('.jpg','') for _f in test_generator.filenames]

In [36]:
predict_index

array([ 9,  4,  6, ..., 10, 11,  7])

In [37]:
fnames[0]

'001d4c8d70ebf7f025fccf256324d3d5ad3560faee1cdf8c7115f5eb033bc3d2'

In [38]:
class_list = [None] * len(train_generator.class_indices)
for _item in train_generator.class_indices:
    class_list[train_generator.class_indices[_item]] = _item

In [39]:
train_generator.class_indices

{'CALsuburb': 0,
 'PARoffice': 1,
 'bedroom': 2,
 'coast': 3,
 'forest': 4,
 'highway': 5,
 'industrial': 6,
 'insidecity': 7,
 'kitchen': 8,
 'livingroom': 9,
 'mountain': 10,
 'opencountry': 11,
 'store': 12,
 'street': 13,
 'tallbuilding': 14}

In [40]:
predict_index.shape

(1500,)

In [41]:
mtype_dict

{'CALsuburb': 9,
 'PARoffice': 7,
 'bedroom': 12,
 'coast': 10,
 'forest': 4,
 'highway': 14,
 'industrial': 2,
 'insidecity': 3,
 'kitchen': 0,
 'livingroom': 5,
 'mountain': 8,
 'opencountry': 6,
 'store': 11,
 'street': 1,
 'tallbuilding': 13}

In [42]:
cateidxs = [mtype_dict[class_list[_p]] for _p in predict_index]

In [43]:
percentile_list = pd.DataFrame(
    {'id': fnames,
     'class': cateidxs
    })

In [44]:
percentile_list.to_csv('out_7.csv',encoding='utf-8', index=False,columns=["id","class"])